In [1]:
using Distances

include("TaxonomyExample.jl")
include("BlahutArimoto.jl")
include("InformationTheoryFunctions.jl")

BAiterations (generic function with 1 method)

In [ ]:

#Setup pre-evalutated utility matrix and the utility-maximum vector
#function setuputilityarrays(a::Vector, o::Vector, utility::Function)



#This function computes p_boltz = 1/Z * p0 * exp(β*ΔU),
#where Z is the normalization constant (partition function)
### arguments:
#p0 ... prior distribution (vector of length N)
#β ... inverse temperature (scalar)
#ΔU ... utility or potential-difference (vector of length N)
### returns:
#p_boltz ... 1/Z * p0 * exp(β*ΔU)
#function boltzmanndist(p0::Vector, β, ΔU::Vector)






#This function performs Blahut-Arimoto iterations for the three-variable general case
function threevarBAiterations(pa_init::Vector, po_init::Vector, pago_init::Matrix,
    β1, β2, β3, U_pre::Matrix, Umax::Vector, pw::Array, ε_conv::Real, maxiter::Integer;
    compute_performance::Bool=false, performance_per_iteration::Bool=false,
    performance_as_dataframe::Bool=false)
    
    pa_new = pa_init
    po_new = po_init
    card_a = size(U_pre,1)
    card_w = size(U_pre,2)
    card_o = size(po_init,1)
    
    #p(a|o)
    pago_new = pago_init
    
    #p(o|w)
    pogw = zeros(card_o,card_w)
    #p(a|o,w)
    pagow = zeros(card_a,card_o,card_w)
    #TODO: Initialize pogw and pagow

    #if performance measures don't need to be returned, don't compute them per iteration
    if compute_performance==false
        performance_per_iteration = false
    end 

    #preallocate if necessary
    if performance_per_iteration 
        I_ow_i = zeros(maxiter)  #I(O;W)
        I_ao_i = zeros(maxiter)  #I(A;O)
        I_awgo_i = zeros(maxiter)  #I(A;W|O)
        Ha_i = zeros(maxiter)  #H(A)
        Ho_i = zeros(maxiter)  #H(O)
        Hago_i = zeros(maxiter)  #H(A|O)
        Hogw_i = zeros(maxiter)  #H(O|W)
        Hagow = zeros(maxiter)  #H(A|O,W)  #TODO: is this really the correct term?
        EU_i = zeros(maxiter)
        RDobj_i = zeros(maxiter)
    end
    
    #main iteration
    iter = 0 #initialize counter, so it persists beyond the loop
    for iter in 1:maxiter
        pa = deepcopy(pa_new)  #make sure not to just copy the reference
        pa_new = zeros(card_a) #TODO: do you need to initialize?
        po = deepcopy(po_new)
        po_new = zeros(card_o) #TODO: do you need to initialize?
        pago = deepcopy(pago_new)
        pago_new = zeros(card_a,card_o) #TODO: do you need to initialize
            
        #compute p(o|w)
        #p(o|w) ∝ p(o) exp( β1 (E[U] - 1/β2 DKL(p(a|o,w)||p(a))) - β1 DKL(p(a|o,w)||p(a|o)) (1/β3-1/β2) )
        #TODO: the implementation below is maybe not that straightforward - make sure it's easily readable!
        
        #------- compute E[U] ---------#
        #TODO: move this into a new function (InformationTheoryFunctions.jl?)                    
        #E[U] = ∑_a,o,w p(w)p(o|w)p(a|o,w) U(a,w)
        #but we can reuse the function that computes E[U] for ∑_a,w p(w)p(a|w) U(a,w)
        #
        #compute p(a|w)
        pagw = zeros(card_a,card_w)
        for j=1:card_w
            #p(a|w) = ∑_o p(o|w)p(a|o,w)
            pagw[:,j] = squeeze(pagow[:,:,j],3) * pogw(:,j)
        end
        EU_i[iter] = expectedutility(pw,pagw,U_pre)
        
        
        #------- compute the two KL terms -----------#
        #TODO: rather than specifying the KL in bits here, do it in InformationTheoryFunctions,
        #simolar to entroybits
        DKL_a = zeros(card_o,card_w)
        DKL_ago = zeros(card_o,card_w)
        
        for j in 1:card_w
            for k in 1:card_o
                DKL_a(k,j) = kl_divergence(vec(pagow[:,k,j]),pa)/log(2) #from package Distances.jl, divide by log(2) for bits
                DKL_ago(k,j) = kl_divergence(vec(pagow[:,k,j]),vec(pago[:,k]))/log(2) #from package Distances.jl, divide by log(2) for bits
            end
        end
        
        pogw_util = EU_i[iter] - 1/β2 * DKL_a - (1/β3-1/β2) * DKL_ago
        #TODO: can you really use EU here, or should the expectation depend on the conditioned vars o,w?
        
        for j in 1:card_w
            #TODO: replace this with the function BoltzmannDist
            pogw_j = po .* exp(β1 * powg_util[:,j])
            pogw[:,j] = pogw_j ./ sum(pogw_j)  #normalize
        end
        
        
        
        
        #compute p(a|o,w)
        #p(a|o,w) ∝ p(a|o) exp( β3 U(a,w) - β3/β2 log(p(a|o)/p(a)) )
        for k in 1:card_o
            for j in 1:card_w
                #TODO: replace this with the function BoltzmannDist
                for a in 1:card_a
                    pagow[i,k,j] = pago[i,k] * exp( β3*U_pre[i,w] - β3/β2*log(pago[i,k]/pa[i])/log(2) #divide by log(2) for bits                    
                end
                #normalize
                pagow[:,k,j] = pagow[:,k,j] / sum(pagow[:,k,j])
            end
        end
        
        
        
        #compute p(o)
        #p(o) = ∑_w p(o|w)p(w)
        po_new = pogw * pw
        
        
        #compute p(a|o)
        #p(a|o) = ∑_w p(w|o)p(a|o,w)   with p(w|o) = p(o|w)p(w)/p(o)
        for k in 1:card_o
            #compute p(w|o=k)
            pwgo_k = vec(pogw[k,:]).*pw / po[k]
            #compute p(a|o=k)
            pago_new[:,k] = squeeze(pagow[:,k,:],2) * pwgo_k
        end
        
        
        
        #compute p(a)
        #p(a) = ∑_w,o p(w)p(o|w)p(a|o,w)
        #compute p(a|w)
        pagw = zeros(card_a,card_w)  #strictly speaking this is not necessary, since we have it from above
        for j=1:card_w
            #p(a|w) = ∑_o p(o|w)p(a|o,w)
            pagw[:,j] = squeeze(pagow[:,:,j],3) * pogw(:,j)
        end
        #p(a) = ∑_w p(a|w)p(w)
        pa_new = pagw * pw

        
        #TODO: pagw is also needed when computing the expected utility above - also, when the expected utility
        #should be computed as a performance measure - it would have to be recomputed with pxx_new
        #-> move the computation of the EU here (before pa_new) and make sure that it is correclty set in
        #the first iteration (since it will be moved to a function, simply calling it there with
        #an if-clause should be fine).
        
        #TODO: is it better to immediately use the pxx_new quantities, or just update all of them
        #after each iteration (the latter is implemented right now)?
        #Does the order of the equations play any role?

        #compute entropic quantities (if requested with additional parameter)
        if performance_per_iteration
            I_i[iter], Ha_i[iter], Hago_i[iter], EU_i[iter], RDobj_i[iter] = analyzeBAsolution(po, pa_new, pago, U_pre, β)
        end

        #check for convergence
        if norm(pa-pa_new) < ε_conv            
            break
        end
    end
    
    #check if iteration limit has been reached (before convergence)
    if iter == maxiter
        warn("[BAiterations] maximum iteration reached - returning... (results might be inaccurate)")
    end



    #return results
    if compute_performance == false
        return pago, vec(pa_new)  #the squeeze will turn pa into a vector again
    else
        if performance_per_iteration == false
            #compute performance measures for final solution
            I, Ha, Hago, EU, RDobj = analyzeBAsolution(po, pa_new, pago, U_pre, β)
        else
            #"cut" valid results from preallocated vector
            I = I_i[1:iter]
            Ha = Ha_i[1:iter]
            Hago = Hago_i[1:iter]
            EU = EU_i[1:iter]
            RDobj = RDobj_i[1:iter]
        end

        #if needed, transform to data frame
        if performance_as_dataframe == false
            return pago, vec(pa_new), I, Ha, Hago, EU, RDobj
        else
            performance_df = performancemeasures2DataFrame(I, Ha, Hago, EU, RDobj)
            return pago, vec(pa_new), performance_df 
        end
    end
    
end


